In [1]:
import os
from woocommerce import API
from dotenv import load_dotenv

load_dotenv()

woocommerce_url = os.getenv('URL')
woocommerce_key = os.getenv('KEY')
woocommerce_secret = os.getenv('SECRET')

wc_api = API(
    url=woocommerce_url,
    consumer_key=woocommerce_key,
    consumer_secret=woocommerce_secret,
    version='wc/v3'
)

In [17]:
response = wc_api.get("products", params={
                     "status": "draft", "per_page": 1, "page": 1})
total_pages = response.headers["X-WP-TotalPages"]
total_products = int(response.headers["X-WP-Total"])


In [18]:
products = []
for page in range(1, int(total_pages) + 1):
    response = wc_api.get("products", params={
                         "status": "draft", "per_page": total_products, "page": page})
    products += response.json()


In [30]:
for product in products:
    if product['sku'].startswith('S'):
        print(product['sku'])

S20756-35
S20838-85
S20756-25
S20755-35
S20755-25
S10756-16
S20762-30
SAW800253-LPRA
SAM800019-BKH
SA81750-MLT
SA81538-VCT
SA81165-VPS
S10392-4
SA81750-RWP
SA81538-BAR
S10293-4
SA81211-CRBH
SA81165-BK
SAW800283-RED
S10737-30
S20813-65
S20759-31
S20688-45
S20688-16
S20687-45
S20681-16
S20678-16
S20723-45
S10684-26
S2108-790
S10650-35
S70555-5
S70555-4
S70532-1
S20654-55
S20654-45
S10654-45
SA81484-BK


In [28]:
out = [product['sku'] for product in products if product['sku'].startswith('S')]

In [29]:
out

['S20756-35',
 'S20838-85',
 'S20756-25',
 'S20755-35',
 'S20755-25',
 'S10756-16',
 'S20762-30',
 'SAW800253-LPRA',
 'SAM800019-BKH',
 'SA81750-MLT',
 'SA81538-VCT',
 'SA81165-VPS',
 'S10392-4',
 'SA81750-RWP',
 'SA81538-BAR',
 'S10293-4',
 'SA81211-CRBH',
 'SA81165-BK',
 'SAW800283-RED',
 'S10737-30',
 'S20813-65',
 'S20759-31',
 'S20688-45',
 'S20688-16',
 'S20687-45',
 'S20681-16',
 'S20678-16',
 'S20723-45',
 'S10684-26',
 'S2108-790',
 'S10650-35',
 'S70555-5',
 'S70555-4',
 'S70532-1',
 'S20654-55',
 'S20654-45',
 'S10654-45',
 'SA81484-BK']

In [34]:
filtered_out = ['S20756-35',
                'S20838-85',
                'S20756-25',
                'S20755-35',
                'S20755-25',
                'S10756-16',
                'S20762-30',
                'SAM800019-BKH',
                'SA81750-MLT',
                'SA81538-VCT',
                'SA81165-VPS',
                'SA81750-RWP',
                'SA81538-BAR',
                'S10293-4',
                'SA81211-CRBH',
                'SA81165-BK',
                'SAW800283-RED',
                'S10737-30',
                'S20813-65',
                'S20759-31',
                'S20688-45',
                'S20688-16',
                'S20687-45',
                'S20681-16',
                'S20678-16',
                'S20723-45',
                'S10684-26',
                'S2108-790',
                'S10650-35',
                'S70555-5',
                'S70555-4',
                'S70532-1',
                'S20654-55',
                'S20654-45',
                'S10654-45',
                'SA81484-BK']


In [31]:
import pandas as pd

In [32]:
df = pd.read_csv('input.csv')

In [35]:
df_filtered = df[df['PARENT SKU'].str.contains('|'.join(filtered_out))]

In [39]:
column_skus = set(df['PARENT SKU'].dropna().astype(str))
missing_skus = set(filtered_out) - column_skus

In [40]:
missing_skus

{'S10756-16',
 'S20755-25',
 'S20755-35',
 'S20756-25',
 'S20756-35',
 'S20762-30',
 'S20838-85'}

In [45]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 28 to 159
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   BREND              29 non-null     object 
 1   PARENT SKU         29 non-null     object 
 2   Bar kod            29 non-null     int64  
 3   Unnamed: 3         29 non-null     object 
 4   Kratki opis        29 non-null     object 
 5   Veličina           29 non-null     float64
 6   Redovna cijena     29 non-null     float64
 7   Snižena            29 non-null     float64
 8   Količina           29 non-null     int64  
 9   Kategorija         29 non-null     object 
 10  Podkategorija      29 non-null     object 
 11  Spol               29 non-null     object 
 12  Podloga            19 non-null     object 
 13  Vrsta proizvoda    29 non-null     object 
 14  Kat.broj za Wand   29 non-null     object 
 15  Vrsta proizvoda.1  18 non-null     object 
dtypes: float64(3), int64(2), object

In [44]:
df_filtered.to_csv('filtered.csv', index=False)

In [50]:
df_filtered = df_filtered.drop(
    columns=['Redovna cijena', 'Podloga', 'Veličina', 'Količina'])


In [55]:
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.lower()
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.replace(
    '[čćžšđ]', lambda x: {'č': 'c', 'ć': 'c', 'ž': 'z', 'š': 's', 'đ': 'd'}[x.group()], regex=True)


In [58]:
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.replace('zenske', 'zenska')
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.replace('muske', 'muska')
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.replace('tenisice', 'tenisica')


In [62]:
df_filtered['Kratki opis'] = df_filtered['Kratki opis'].str.replace(' ', '-')
df_filtered['Unnamed: 3'] = df_filtered['Unnamed: 3'].str.replace(' ', '-')

In [65]:
df_filtered['out'] = pd.concat(
    [df_filtered['Unnamed: 3'], df_filtered['Kratki opis'], df_filtered['PARENT SKU']], axis=1, join='inner').apply('-'.join, axis=1)

In [67]:
sku_dict = df_filtered.set_index('PARENT SKU')['out'].to_dict()

In [70]:
sku_dict

{'SA81484-BK': 'SAUCONY-Scoot-Capri-zenska-kratke-tajice-SA81484-BK',
 'S10654-45': 'SAUCONY-Guide-14-zenska-tenisica-za-trcanje-S10654-45',
 'S20654-45': 'SAUCONY-Guide-14-muska-tenisica-za-trcanje-S20654-45',
 'S20654-55': 'SAUCONY-Guide-14-muska-tenisica-za-trcanje-S20654-55',
 'S70532-1': 'SAUCONY-Grid-Web-unisex-tenisica-S70532-1',
 'S70555-4': 'SAUCONY-Jazz-Court-unisex-tenisica-S70555-4',
 'S70555-5': 'SAUCONY-Jazz-Court-unisex-tenisica-S70555-5',
 'S10650-35': 'SAUCONY-Ride-14-zenska-tenisica-za-trcanje-S10650-35',
 'S2108-790': 'SAUCONY-Shadow-Original-muska-tenisica-S2108-790',
 'S10684-26': 'SAUCONY-Guide-15-zenska-tenisica-za-trcanje-S10684-26',
 'S20723-45': 'SAUCONY-Kinvara-13-muska-tenisica-za-trcanje-S20723-45',
 'S20678-16': 'SAUCONY-Triumph-19-muska-tenisica-za-trcanje-S20678-16',
 'S20681-16': 'SAUCONY-Omni-20-muska-tenisica-za-trcanje-S20681-16',
 'S20687-45': 'SAUCONY-Endorphin-Pro-2-muska-tenisica-za-trcanje-S20687-45',
 'S20688-16': 'SAUCONY-Endorphin-Speed-2-mus